In [ ]:
# Diogo Meneses Franco
# RA:2202455 

In [ ]:
%%sh
sudo pip install spark
sudo pip install pyspark

In [1]:
import pandas as pd

#Importando o spark e o pyspark
import spark,pyspark

#Importando as bibliotecas do pyspark.sql 
from pyspark.sql import *

#Importando as funções sql do spark
#documentação https://spark.apache.org/docs/latest/api/sql/index.html
from pyspark.sql import functions as f

#Importando os tipos de dados do spark
#documentação https://spark.apache.org/docs/latest/sql-ref-datatypes.html
from pyspark.sql import types as t 


#Biblioteca datetime
from datetime import datetime, date

In [2]:
#Criando uma Sessão do Spark (Spark Session)
spark = SparkSession.builder.master("local").appName("AC5 DataEng").getOrCreate()

In [ ]:
#Criar um dataframe lendo o arquivo vendas.parquet

df = spark.read.format("parquet").load("/content/vendas.parquet")
df.show()

In [ ]:
#Converter a coluna data para o tipo DateType.
df = df.withColumn("data",df.data.cast(t.DateType()))
df.show()

In [ ]:
#Criar as colunas, utilizando as funções spark.sql.functions.year, spark.sql.functions.month , spark.sql.functions.dayofMonth:
df = df.withColumn("ano",f.year("data"))
df = df.withColumn("Mes",f.month("data"))
df = df.withColumn("Dia",f.dayofmonth("data"))
df.show()

In [ ]:
#Na coluna sms, se caso o valor for null , troque por False (use a função spark.sql.functions.when (df.sms.isNull(),False).otherwise(df.sms)) 
df.select(df.sms,f.when(df.sms.isNull(),False).otherwise(df.sms)).show()

In [ ]:
#Crie uma coluna chamada “total” que deve ser a multiplicação das colunas “vlr” e “qtd”.
df = df.withColumn("total",df.vlr * df.qtd)
df.show()


In [8]:
#Grave o dataframe particionado por ano, mes e dia no formato “parquet” de nome “vendas_tratadas”.
df.write.format("parquet").mode("overwrite").partitionBy("ano","mes","dia").save("vendas_tratadas")


In [ ]:
#Crie uma tabela temporaria chamada “vendas_table” (df.registerTempTable(“vendas_table”))
df.registerTempTable("vendas_table")


In [ ]:
#Crie um dataframe spark.sql utilizando a query spark.sql(“select distinct cidade from vendas_table”). Grave esse dataframe no formato “parquet” de nome  “cidade”
df_cid = spark.sql("select distinct cidade from vendas_table")
df_cid.write.format("parquet").mode("overwrite").save("cidade")
df_cid.show()



In [ ]:
#Crie um novo dataframe chamado “produto” selecionando as colunas: “des_produto”,”des_familia”, “des_secao”, “des_categoria”,”des_sub_categoria”. Remova possíveis espaços no começo e no final das colunas (spark.sql.functions.trim). Deixe apenas valores únicos (df.distinct()). Grave esse dataframe como parquet “produto”.

df_prod = spark.sql("select distinct des_produto, des_familia, des_secao, des_categoria, des_sub_categoria from vendas_table")
df_prod.show()

In [ ]:
for colunas in df_prod.columns:
    df_prod = df_prod.withColumn(colunas, f.trim(f.col(colunas)))
df_prod.show()

In [13]:
df.write.format("parquet").mode("overwrite").save("produto")


In [ ]:
#Crie um dataframe chamado “venda_mulheres”, selecionando todas as colunas e filtrando apenas vendas realizadas pelo sexo “F”, grave esse dataframe no formato json “venda_mulher”. r
df_f = spark.sql("select * from vendas_table")
df_f.show()


In [ ]:
df_f = df_f.filter(df_f.sexo=='F')
df_f.show()

In [27]:
df_f.write.format("json").mode("overwrite").save("venda_mulher")

In [ ]:
#Crie um dataframe chamado “venda_unicas”, selecionando todas as colunas e filtrando apenas vendas com a qtd igual 1 e o vlr menor que 10. Grave esse dataframe no formato csv “venda_unica”. (df.write.format(“csv”).save(“venda_unica”)

df_vu = spark.sql("select * from vendas_table")
df_vu = df_vu.filter("qtd==1 and vlr <10")
df_vu.show()


In [42]:
df_vu.write.format("csv").mode("overwrite").save("venda_unica")